In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/ProtonX/Kaggle 02

/content/drive/My Drive/ProtonX/Kaggle 02


In [ ]:
ls

 Covid-19/                       __MACOSX/      sm13.csv
 Covid-19.zip                    pre-train.h5   sm1.csv
'Job done!  - Kaggle 02.ipynb'   sm10.csv       train_drop02.h5
 lr0.0001-eps500.h5              sm11.csv       train.h5
 lr0.001-RMS-eps50.h5            sm12.csv


In [ ]:
#!unzip -qq Covid-19.zip

In [ ]:
train_folder='./Covid-19/Train/'

In [ ]:
path_file_labels = './Covid-19/train.csv'

In [ ]:
test_folder = './Covid-19/Test/'

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import imagenet_utils

In [ ]:
labels = pd.read_csv(path_file_labels)
labels.head()

,image,label
0,ProtonX-1.jpg,1
1,ProtonX-2.jpg,1
2,ProtonX-3.jpg,0
3,ProtonX-4.jpg,0
4,ProtonX-5.jpg,1


In [ ]:
def load_images_from_folder(folder, mapping_labels):
    new_width = 224
    new_height = 224
    # threshold1 = 50
    # threshold2 = 220

    images = []
    labels = []
    for filename in os.listdir(folder):
      path_img = folder + filename

      #Đọc ảnh từ folder
      img = load_img(path_img, target_size=(224, 224))
      img = img_to_array(img) *1.0 / 255

      img = np.expand_dims(img, 0)

      #Lấy nhãn tương ứng
      labels.append(int(mapping_labels.loc[(mapping_labels['image']==filename), 'label']))
        
      if img is not None:
        images.append(img)
    return np.vstack(images), np.array(labels)

In [ ]:
image, label = load_images_from_folder(train_folder, labels)

In [ ]:
image.shape

(251, 224, 224, 3)

In [ ]:
 label.shape

(251,)

In [ ]:
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop

In [ ]:
#Trộn lại tất cả các ảnh
image, label = shuffle(image, label)

In [ ]:
#Chia train va validation
X_train, X_val, y_train, y_val = train_test_split(image, label, test_size=0.2, random_state=42)

In [ ]:
# baseModel = VGG16(weights='imagenet', include_top=False,
#                   input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# # ADD some layers
# # Lấy output của ConvNet trong VGG16
# fcHead = baseModel.output

# #Flatten trước khi dùng FC
# fcHead = Flatten(name='flatten')(fcHead)

# #Thêm FC
# fcHead = Dense(256, activation='relu')(fcHead)
# fcHead = Dropout(0.5)(fcHead)

# fcHead = Dense(3, activation='softmax')(fcHead)

In [ ]:
#model = model = Model(inputs=baseModel.input, outputs=fcHead)



In [ ]:
#augmentation cho training data
#augmentation cho training data
# aug_train = ImageDataGenerator( rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
#                          zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# # augementation cho test
# aug_test= ImageDataGenerator()

In [ ]:
# # freez VGG model
# for layer in baseModel.layers:
#   layer.trainable = False

# opt = RMSprop(0.001)
# model.compile(opt, 'sparse_categorical_crossentropy', ['accuracy'])
# numOfEpoch = 25
# H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
#                         steps_per_epoch=len(X_train)//32,
#                         validation_data=(aug_test.flow(X_val, y_val, batch_size=32)),
#                         validation_steps=len(X_val)//32,
#                         epochs=numOfEpoch)


In [ ]:
# # unfreeze some last CNN layer:
# for layer in baseModel.layers[15:]:
#     layer.trainable = True

# numOfEpoch = 35
# opt = SGD(0.001)
# model.compile(opt, 'sparse_categorical_crossentropy', ['accuracy'])
# H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
#                         steps_per_epoch=len(X_train)//32,
#                         validation_data=(aug_test.flow(X_val, y_val, batch_size=32)),
#                         validation_steps=len(X_val)//32,
#                         epochs=numOfEpoch)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3, 3), padding='same', activation='sigmoid'))

In [ ]:
model.add(BatchNormalization())

In [ ]:
model.add(MaxPooling2D(2, 2))

In [ ]:
model.add(Conv2D(32, (3, 3), padding='same', activation='sigmoid'))

In [ ]:
model.add(MaxPooling2D(2, 2))

In [ ]:
model.add(Flatten(input_shape=(224, 224, 3)))

In [ ]:
model.add(Dense(128, activation='sigmoid'))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(3, activation='softmax'))

In [ ]:
opt = SGD(0.001)

In [ ]:
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 300

In [ ]:
model.fit(X_train, y_train, epochs=epochs, verbose=1, validation_data=(X_val, y_val))#verbose in ra qtrinh train

Epoch 1/300
7/7 [==============================] - 0s 68ms/step - loss: 1.1643 - accuracy: 0.3750 - val_loss: 1.0599 - val_accuracy: 0.5098
Epoch 2/300
7/7 [==============================] - 0s 33ms/step - loss: 1.1404 - accuracy: 0.3900 - val_loss: 1.2937 - val_accuracy: 0.1961
Epoch 3/300
7/7 [==============================] - 0s 32ms/step - loss: 1.1269 - accuracy: 0.3650 - val_loss: 1.0211 - val_accuracy: 0.5098
Epoch 4/300
7/7 [==============================] - 0s 33ms/step - loss: 1.0494 - accuracy: 0.4650 - val_loss: 1.1625 - val_accuracy: 0.1961
Epoch 5/300
7/7 [==============================] - 0s 33ms/step - loss: 1.0365 - accuracy: 0.5100 - val_loss: 1.0421 - val_accuracy: 0.5098
Epoch 6/300
7/7 [==============================] - 0s 34ms/step - loss: 1.0055 - accuracy: 0.5100 - val_loss: 1.1481 - val_accuracy: 0.1961
Epoch 7/300
7/7 [==============================] - 0s 34ms/step - loss: 1.0422 - accuracy: 0.4650 - val_loss: 1.2108 - val_accuracy: 0.1961
Epoch 8/300
7/7 [===

In [ ]:
# H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
#                         steps_per_epoch=len(X_train)//32,
#                         validation_data=(aug_test.flow(X_val, y_val, batch_size=32)),
#                         validation_steps=len(X_val)//32,
#                         epochs=epochs)

In [ ]:
#name_model = str(epochs)+str()

In [ ]:
model.save('train_drop02.h5')

ValueError: ignored

# Predict

In [ ]:
import tensorflow as tf

In [ ]:
save_model = tf.keras.models.load_model('train_drop02.h5')

In [ ]:
test = pd.read_csv('./Covid-19/sample_submission.csv')

In [ ]:
submit = pd.DataFrame(columns=['image1','label1'])
for filename in os.listdir(test_folder):
    path_img = test_folder + filename

    #Đọc ảnh từ folder
    img = load_img(path_img, target_size=(224, 224))
    img = img_to_array(img) *1.0 / 255
    img = np.expand_dims(img, 0) 
    img = np.vstack([img])

    pred = np.argmax(save_model.predict(img, batch_size=10))

    submit = submit.append({'image1':filename, 'label1':pred}, ignore_index=True)

In [ ]:
submit["key"] = submit["image1"].apply(lambda x: x[8:])
submit['key'] = submit['key'].replace('\.jpg','',regex=True).astype(float)

In [ ]:
submit.head()

,image1,label1,key
0,ProtonX-48.jpg,1,48.0
1,ProtonX-3.jpg,0,3.0
2,ProtonX-53.jpg,0,53.0
3,ProtonX-34.jpg,0,34.0
4,ProtonX-11.jpg,1,11.0


In [ ]:
test["key"] = test["image"].apply(lambda x: x[8:])
test['key'] = test['key'].replace('\.jpg','',regex=True).astype(float)

KeyError: ignored

In [ ]:
sub = pd.merge(test, submit)
sub

In [ ]:
submit1 = sub.loc[:,'key']

# Submit

In [ ]:
final = pd.read_csv('./Covid-19/sample_submission.csv')

In [ ]:
final['label'] = submit1

In [ ]:
final

In [ ]:
final.to_csv('sm13.csv', index=False)